In [1]:
%load_ext autoreload
%autoreload 2
#from gaia import *
import glob, os

import numpy as np
import IPython
from ipywidgets import *
import tkinter as tk
from tkinter import filedialog
import warnings
warnings.filterwarnings('ignore')

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import tensorflow as tf

import keras
print(keras.__version__)
print(tf.__version__)

from keras.layers import Activation, Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Cropping2D, Cropping3D, Deconvolution2D, Flatten, Add, BatchNormalization
from keras.models import Model
from keras import backend as K
from keras.callbacks import TensorBoard, LearningRateScheduler
from keras.utils import multi_gpu_model 
from keras.callbacks import History 
from keras.models import load_model
from keras.regularizers import l2

from IPython.display import HTML
from itertools import chain
import random

#from mpl_toolkits.mplot3d import Axes3D
import pickle
import re
from joblib import dump, load, Parallel, delayed

#%matplotlib notebook
#%matplotlib notebook 
%matplotlib inline
from matplotlib import ticker
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.tri as tri
font = {'size'   : 18}
matplotlib.rc('font', **font)
from matplotlib import rcParams

import time
import sklearn
from time import sleep 
from sklearn.utils import shuffle
from multiprocessing import Process, Queue
import threading

from my_classes import DataGenerator
from my_classes import PatchedModelCheckpoint

Using TensorFlow backend.


2.2.4
1.15.0


In [2]:
#pathSave = "C:/Users/agsid/Google Drive/ML/"
pathSave = "/zeus/zeususer/agar_si/gaia/python/Compressed_Forward_Mars/"

In [3]:
class Regression:
    def __init__(self,_data, _pathSave, _numChannels, _sVar):
        self.pathSave = _pathSave
        self.sVar = _sVar

        self.x = _data.x
        self.y = _data.y
        
        self.numChannels = _numChannels
        self.numFolders_tr = 8048
        self.numFolders_val = 441
        self.xdim0 = 302
        self.xdim1 = 394
        
        tf.keras.backend.clear_session()
        
    def NN(self, _hSize, _NEPOCH, _learnRate, _mpDictTrain={}, _mpDictTest={}):
        graph = tf.get_default_graph()
        input_img = Input(shape=(self.xdim0, self.xdim1, self.numChannels))  
        # adapt this if using `channels_first` image data format
        l2reg = l2(1e-7)
        activFunc = 'tanh'
        fx = 5
        fy = 5
        sx = 3
        sy = 3
        x = Conv2D(3, (fx, fy), strides=(sx,sy), padding='same', activation=activFunc, kernel_regularizer=l2reg)(input_img)
        x = BatchNormalization()(x)
        x = Conv2D(6, (fx, fy), strides=(sx,sy), padding='same', activation=activFunc, kernel_regularizer=l2reg)(x)
        x = BatchNormalization()(x)
        x = Conv2D(9, (fx, fy), strides=(sx,sy), padding='same', activation=activFunc, kernel_regularizer=l2reg)(x)
        encoded = BatchNormalization()(x)
        
        x = Deconvolution2D(6, (fx, fy), strides=(sx,sy), padding='same', activation=activFunc, kernel_regularizer=l2reg)(encoded)
        x = Cropping2D(cropping=((1, 1), (1, 0)), data_format=None)(x)
        x = BatchNormalization()(x)
        
        x = Deconvolution2D(3, (fx, fy), strides=(sx,sy), padding='same', activation=activFunc, kernel_regularizer=l2reg)(x)
        x = Cropping2D(cropping=((1, 0), (0, 0)), data_format=None)(x)
        x = BatchNormalization()(x)
        
        x = Deconvolution2D(1, (fx, fy), strides=(sx,sy), padding='same', activation=activFunc, kernel_regularizer=l2reg)(x)
        x = Cropping2D(cropping=((0, 1), (1, 1)), data_format=None)(x)   
        x = BatchNormalization()(x)
        
        decoded = Conv2D(self.numChannels, (fx, fy), strides=(1,1), padding='same', activation=activFunc, kernel_regularizer=l2reg)(x)
        
        paramsTrain = {'trainOrVal': 'train',
                       'batch_size': 64,
                       'xdim': (self.xdim0,self.xdim1), 
                       'ydim': (self.xdim0,self.xdim1),
                       'n_channels': self.numChannels,
                       'shuffle': True, 
                       'isAutoencoder': True}
                        
        paramsVal = {'trainOrVal': 'cv',
                       'batch_size': 64,
                       'xdim': (self.xdim0,self.xdim1), 
                       'ydim': (self.xdim0,self.xdim1),
                       'n_channels': self.numChannels,
                       'shuffle': True, 
                       'isAutoencoder': True}
        
        validation_generator = DataGenerator(self.pathSave, self.sVar, self.numFolders_val, **paramsVal)
        training_generator = DataGenerator(self.pathSave, self.sVar, self.numFolders_tr, **paramsTrain)
        
        class TimeHistory(keras.callbacks.Callback):
            def on_train_begin(self, logs={}):
                self.times = []

            def on_epoch_begin(self, batch, logs={}):
                self.epoch_time_start = time.time()

            def on_epoch_end(self, batch, logs={}):
                self.times.append(time.time() - self.epoch_time_start)
        
        class LossPrintingCallback(tf.keras.callbacks.Callback):
            def on_epoch_end(self, epoch, logs=None):
                if epoch%1000==0:
                    print('learning rate is {:.16e}'.format(tf.keras.backend.get_value(self.model.optimizer.lr)))
                    
        def scheduler(epoch):
            initial_lrate = _learnRate
            if epoch < 1:
                return initial_lrate
            elif epoch >= 1 and epoch < 2:
                return initial_lrate/10.
            elif epoch >= 2 and epoch < 3:
                return initial_lrate/100.
            else:
                return initial_lrate/1000.
        
        def root_mean_squared_error(y_true, y_pred):
            return K.sqrt(K.mean(K.square(y_pred - y_true))) 
        
        config = tf.ConfigProto(
        #device_count={'GPU': 1},
        intra_op_parallelism_threads=1,
        allow_soft_placement=True)

        config.gpu_options.allow_growth = True
        config.gpu_options.per_process_gpu_memory_fraction = 0.8

        session = tf.Session(config=config)
        
        GPUS = 1
        name = self.sVar + '_Mars300_sims10k_f5s3_c369_tanh_l2reg_wlrsc'
        
        with session.as_default():
            with session.graph.as_default():
                if GPUS >= 2:
                    with tf.device("/cpu:0"):
                        # initialize the model
                        autoencoder = Model(input_img, decoded)

                    # make the model parallel
                    autoencoder = multi_gpu_model(autoencoder, gpus=GPUS)
                else:
                    autoencoder = Model(input_img, decoded)
                    #autoencoder = load_model(self.pathSave + "/TrainedNetworks/autoencoder/" + name + ".hdf5")
                autoencoder.compile(optimizer=keras.optimizers.Adam(), loss='mse') #lr=_learnRate, amsgrad=True
                autoencoder.summary()
                
                K.get_session().run(tf.global_variables_initializer())
                
                history = History()
                cp = PatchedModelCheckpoint(self.pathSave + 'TrainedNetworks/autoencoder/' + name + '.{epoch:02d}-{val_loss:.5f}.hdf5', 
                monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)
                time_callback = TimeHistory()
                lr_scheduler = LearningRateScheduler(scheduler)
                csvlogger = tf.keras.callbacks.CSVLogger(self.pathSave+'TrainedNetworks/autoencoder/'+name+'.txt', separator=",", append=True)
                
                ae = autoencoder.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=16,
                    verbose=1,
                    max_queue_size=100,                 
                    epochs=100,
                    callbacks=[cp, csvlogger, lr_scheduler]) #, TensorBoard(log_dir= self.pathSave + '/TrainedNetworks/autoencoder')])
                
                #with open(self.pathSave + 'TrainedNetworks/autoencoder/History_' + name + '.txt', 'wb') as file_pi:
                #    dump(ae.history, file_pi)
                #autoencoder.save(self.pathSave + 'TrainedNetworks/autoencoder/my_model.h5')
        
        return self 

In [4]:
## NN parameters
nnDict = {}
counter = 0
Variables = ['T']
nnDict["Architecture" + str(counter)] = [1000,500,1000]
counter += 1
nnCounter = counter
numChannels = 1

resultsDict = {}
for sVar in Variables:
    dictToOpen = "processedDicts/Mars_new/ConvAE/" + sVar + "/Dict_processed_data_MarsNew_2D_ConvAE_" + sVar + "_[12000]sims_1channel.txt" 
    with open(pathSave + dictToOpen, "rb") as myFile:
        Dict_processed_data = load(myFile)
        
    for i in range(0,nnCounter):

        class createData():
            pass
        data = createData()
        #data.x_data = Dict_processed_data_train #['x_data']
        #data.x_cv = Dict_processed_data_cv #['x_cv']
        data.x = Dict_processed_data['x'] 
        data.y = Dict_processed_data['y'] 
        
        R1 = Regression(data, pathSave, numChannels, sVar)
        results = R1.NN(nnDict["Architecture" + str(i)], 5, 0.001)

        #resultsDict["loss_train"] = results.Loss
        #resultsDict["loss_test"]  = results.Loss_cv
        #resultsDict["x_data"]     = results.x_data
        #resultsDict["x_test"]     = results.x_test
        #resultsDict["x_cv"]       = results.x_cv

        print()
        #with open(pathSave + "TrainedNetworks/" + "/resultsDict" + str(nnDict["Architecture" + str(i)]) + ".txt", "wb") as fkl:
        #    pickle.dump(resultsDict, fkl, protocol=4)

W1010 18:33:19.183356 139830479693632 module_wrapper.py:139] From /zeus/zeus_scr/agar_si/python/anaconda3/envs/tf_gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1010 18:33:19.185714 139830479693632 module_wrapper.py:139] From /zeus/zeus_scr/agar_si/python/anaconda3/envs/tf_gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1010 18:33:19.213753 139830479693632 module_wrapper.py:139] From /zeus/zeus_scr/agar_si/python/anaconda3/envs/tf_gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W1010 18:33:19.214518 139830479693632 module_wrapper.py:139] From /zeus/zeus_scr/agar_si/python/anaconda3/envs/tf_gpu/lib/python3.7/site-packages/keras/backend/tensorflow_

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 302, 394, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 101, 132, 3)       78        
_________________________________________________________________
batch_normalization_1 (Batch (None, 101, 132, 3)       12        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 44, 6)         456       
_________________________________________________________________
batch_normalization_2 (Batch (None, 34, 44, 6)         24        
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 15, 9)         1359      
_________________________________________________________________
batch_normalization_3 (Batch (None, 12, 15, 9)         36        
__________

W1010 18:33:33.858285 139830479693632 module_wrapper.py:139] From /zeus/zeus_scr/agar_si/python/anaconda3/envs/tf_gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W1010 18:33:34.023474 139830479693632 module_wrapper.py:139] From /zeus/zeus_scr/agar_si/python/anaconda3/envs/tf_gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



Epoch 1/100


FailedPreconditionError: 2 root error(s) found.
  (0) Failed precondition: Attempting to use uninitialized value conv2d_1/bias
	 [[{{node conv2d_1/bias/read}}]]
	 [[loss/add_6/_327]]
  (1) Failed precondition: Attempting to use uninitialized value conv2d_1/bias
	 [[{{node conv2d_1/bias/read}}]]
0 successful operations.
0 derived errors ignored.